In [1]:
# force geopandas to use shapely instead of pygeos
import os
os.environ['USE_PYGEOS'] = '0'

import numpy as np
import pandas as pd
import plotly.express as px
from shapely import wkt
from shapely.geometry import Point
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
meters = pd.read_csv("../data/Parking_Meters.csv")
#street = pd.read_csv("../data/street_sweeping.csv")

In [6]:
meters['shape'] = meters['shape'].apply(wkt.loads)
meters = gpd.GeoDataFrame(meters, geometry='shape')
meters.crs = 'EPSG:4326'

street_df = pd.read_csv('../data/street_sweeping.csv')
street_df = street_df[~street_df.Line.isnull()]
street_df.loc[:, 'Line'] = street_df.Line.apply(wkt.loads)
street_df = gpd.GeoDataFrame(street_df, geometry='Line')
street_df.crs = 'EPSG:4326'

unique_streets = street_df.drop_duplicates(subset='Line')

In [7]:
joined = gpd.sjoin_nearest(meters, unique_streets, distance_col='dist', how='left')
joined.shape

/Users/tessa/opt/anaconda3/envs/stat-222/lib/python3.9/site-packages/geopandas/array.py:344: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(


(35645, 63)

In [14]:
dups = meters.drop_duplicates(subset="shape")

In [17]:
meters[~meters.OBJECTID.isin(dups.OBJECTID)]

,OBJECTID,PARKING_SPACE_ID,POST_ID,MS_PAY_STATION_ID,MS_SPACE_NUM,SENSOR_FLAG,ON_OFFSTREET_TYPE,OSP_ID,JURISDICTION,PM_DISTRICT_ID,...,NFC_KEY,SPT_CODE,COLLECTION_ROUTE_DESC,COLLECTION_SUBROUTE_DESC,shape,Neighborhoods,SF Find Neighborhoods,Current Police Districts,Current Supervisor Districts,Analysis Neighborhoods
2816,5269096,0,701-02240,-,0,Y,ON,0.0,SFMTA,3.0,...,4Z9Y,NaN,Van Ness Ave: 200 - 2100,Van Ness,POINT (-122.41967 37.77802),21,21,4,10,36


In [37]:
active = meters[(meters["ACTIVE_METER_FLAG"] == "M") | (meters["ACTIVE_METER_FLAG"] == "T")]

In [38]:
active.shape

(29313, 45)

In [39]:
dups = active.drop_duplicates(subset="shape")

In [40]:
dups.shape

(29288, 45)

In [45]:
active[~active.OBJECTID.isin(dups.OBJECTID)]

active[active["shape"] == active.iloc[16237]["shape"]]['ACTIVE_METER_FLAG']

19768    T
Name: ACTIVE_METER_FLAG, dtype: object

In [46]:
active[~active.OBJECTID.isin(dups.OBJECTID)]['ACTIVE_METER_FLAG']

16237    M
19791    T
20158    M
22371    M
25472    T
26541    M
28144    M
28382    M
29514    M
30264    M
30384    M
30684    T
30865    M
30884    M
31288    M
31381    M
31939    M
32430    M
32763    M
33060    M
33867    M
33941    M
34064    M
34276    M
34569    M
Name: ACTIVE_METER_FLAG, dtype: object

In [47]:
joined = gpd.sjoin_nearest(active, unique_streets, distance_col='dist', how='left')
joined.shape

/Users/tessa/opt/anaconda3/envs/stat-222/lib/python3.9/site-packages/geopandas/array.py:344: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(


(29334, 63)

In [51]:
joined['street_id'] = joined['Corridor'] + ' ' + joined['Limits']

In [60]:
joined.groupby("street_id").size()

street_id
01st St Clementina St  -  Folsom St               4
01st St Elim St  -  Mission St                    6
01st St Folsom St  -  Guy Pl                     11
01st St Guy Pl  -  Lansing St                    11
01st St Howard St  -  Tehama St                  15
                                                 ..
Wilmot St Webster St  -  Fillmore St              1
Woodacre Dr Ocean Ave  -  Junipero Serra Blvd     1
York St 23rd St  -  24th St                       6
York St 24th St  -  25th St                       7
Zoe St Bryant St  -  Welsh St                     1
Length: 2113, dtype: int64

/Users/tessa/opt/anaconda3/envs/stat-222/lib/python3.9/site-packages/geopandas/geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [57]:
unique_streets

,CNN,Corridor,Limits,CNNRightLeft,BlockSide,FullName,WeekDay,FromHour,ToHour,Week1,...,Holidays,BlockSweepID,Line,X1,Y1,X2,Y2,Endpoint1,Endpoint2,segment_id
0,9102000,Mission St,Julia St - 08th St,L,SouthEast,Thursday,Thu,2,6,1,...,1,1618894,"LINESTRING (-122.41241 37.77805, -122.41316 37...",-122.412413,37.778045,-122.413158,37.777457,-122.412413343787 37.778045328617,-122.413157691449 37.777457419935,Mission St Julia St - 08th St
1,1191000,22nd St,Mission St - Bartlett St,L,South,Friday,Fri,6,8,1,...,0,1627862,"LINESTRING (-122.41875 37.75544, -122.41986 37...",-122.418748,37.755437,-122.419858,37.755366,-122.418747573844 37.75543693624,-122.419857942892 37.755365729245,22nd St Mission St - Bartlett St
2,4331000,Congdon St,Maynard St - Silver Ave,L,East,Wednesday,Wed,11,13,1,...,0,1637394,"LINESTRING (-122.42602 37.72940, -122.42596 37...",-122.426025,37.729397,-122.425964,37.728641,-122.42602470434 37.72939698234,-122.42596370352 37.72864102171,Congdon St Maynard St - Silver Ave
3,12043000,Sparta St,Ankeny St - Harkness Ave,R,West,Friday,Fri,12,15,1,...,0,1611118,"LINESTRING (-122.40355 37.71834, -122.40347 37...",-122.403555,37.718337,-122.403470,37.718197,-122.403554923986 37.718336652479,-122.403470376794 37.718196907433,Sparta St Ankeny St - Harkness Ave
4,7602000,Jones St,Beach St - Jefferson St,R,East,Thursday,Thu,2,6,1,...,0,1616072,"LINESTRING (-122.41726 37.80707, -122.41745 37...",-122.417261,37.807073,-122.417446,37.808013,-122.417260631616 37.80707251417,-122.41744593863 37.808013324233,Jones St Beach St - Jefferson St
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37676,8349101,Lincoln Way,11th Ave - 12th Ave,L,South,Tue 1st & 3rd,Tues,10,12,1,...,0,1639069,"LINESTRING (-122.46858 37.76583, -122.46870 37...",-122.468583,37.765830,-122.468704,37.765778,-122.468582998602 37.765830028099,-122.468704484779 37.765777895744,Lincoln Way 11th Ave - 12th Ave
37691,2163101,Alemany Blvd,Theresa St - Cotter St,L,SouthEast,Friday,Fri,7,9,1,...,0,1613201,"LINESTRING (-122.43448 37.72873, -122.43448 37...",-122.434479,37.728732,-122.434479,37.728633,-122.434478649123 37.728731782781,-122.434478621488 37.728633080917,Alemany Blvd Theresa St - Cotter St
37761,9650101,Noriega St,16th Ave - 17th Ave,L,South,Wed 1st & 3rd,Wed,11,13,1,...,0,1615535,"LINESTRING (-122.47326 37.75438, -122.47338 37...",-122.473262,37.754381,-122.473380,37.754328,-122.473262412374 37.754381155281,-122.47337999393 37.754328378385,Noriega St 16th Ave - 17th Ave
37796,4782101,Diamond Heights Blvd,Diamond St - Berkeley Way,L,South,Thu 2nd & 4th,Thu,10,12,0,...,0,1628325,"LINESTRING (-122.43740 37.73829, -122.43747 37...",-122.437398,37.738288,-122.437470,37.738266,-122.43739820428 37.738287969238,-122.437470081626 37.738265757979,Diamond Heights Blvd Diamond St - Berkeley Way


In [62]:
pay = pd.read_parquet("../data/meter_data_2022.parquet")

In [63]:
pay.head()

,TRANSMISSION_DATETIME,POST_ID,STREET_BLOCK,PAYMENT_TYPE,SESSION_START_DT,SESSION_END_DT,METER_EVENT_TYPE,GROSS_PAID_AMT
45536904,222408714_4_01012022002410,831-02200,EMBARCADERO SOUTH 200,CASH,2022/01/01 12:24:10 AM,2022/01/01 12:24:10 AM,NS,1.50
45536905,222408715_4_01012022003157,546-28060,LEAVENWORTH ST 2800,CASH,2022/01/01 12:31:57 AM,2022/01/01 12:31:57 AM,NS,0.50
45536906,222408723_4_01012022011253,830-03290,EMBARCADERO NORTH 300,CASH,2022/01/01 01:12:53 AM,2022/01/01 01:12:53 AM,NS,0.50
45536907,222408728_4_01012022020117,336-14220,BUSH ST 1400,CASH,2022/01/01 02:01:17 AM,2022/01/01 02:01:17 AM,NS,0.25
45536908,222408729_4_01012022022638,568-05550,MISSION ST 500,CASH,2022/01/01 02:26:38 AM,2022/01/01 02:26:38 AM,NS,1.00


In [65]:
df = pd.merge(pay, active, on='POST_ID')

In [66]:
df.head()

,TRANSMISSION_DATETIME,POST_ID,STREET_BLOCK,PAYMENT_TYPE,SESSION_START_DT,SESSION_END_DT,METER_EVENT_TYPE,GROSS_PAID_AMT,OBJECTID,PARKING_SPACE_ID,...,NFC_KEY,SPT_CODE,COLLECTION_ROUTE_DESC,COLLECTION_SUBROUTE_DESC,shape,Neighborhoods,SF Find Neighborhoods,Current Police Districts,Current Supervisor Districts,Analysis Neighborhoods
0,222408714_4_01012022002410,831-02200,EMBARCADERO SOUTH 200,CASH,2022/01/01 12:24:10 AM,2022/01/01 12:24:10 AM,NS,1.50,5272708,0,...,71ME,NaN,South Embarcadero,Embarcadero South/Howard/Steuart | Unit-300/Un...,POINT (-122.39102 37.79161),108,108,1,10,8
1,222412313_4_01022022121822,831-02200,EMBARCADERO SOUTH 200,CREDIT CARD,2022/01/02 12:18:22 PM,2022/01/02 01:18:22 PM,NS,8.00,5272708,0,...,71ME,NaN,South Embarcadero,Embarcadero South/Howard/Steuart | Unit-300/Un...,POINT (-122.39102 37.79161),108,108,1,10,8
2,222412993_4_01022022135210,831-02200,EMBARCADERO SOUTH 200,CREDIT CARD,2022/01/02 01:52:10 PM,2022/01/02 03:52:17 PM,NS,14.49,5272708,0,...,71ME,NaN,South Embarcadero,Embarcadero South/Howard/Steuart | Unit-300/Un...,POINT (-122.39102 37.79161),108,108,1,10,8
3,222448944_4_01032022140304,831-02200,EMBARCADERO SOUTH 200,CREDIT CARD,2022/01/03 02:03:04 PM,2022/01/03 04:23:28 PM,NS,9.06,5272708,0,...,71ME,NaN,South Embarcadero,Embarcadero South/Howard/Steuart | Unit-300/Un...,POINT (-122.39102 37.79161),108,108,1,10,8
4,832929888_9_01032022180700,831-02200,EMBARCADERO SOUTH 200,PAY BY CELL,2022/01/03 06:07:00 PM,2022/01/03 08:07:00 PM,NS,4.00,5272708,0,...,71ME,NaN,South Embarcadero,Embarcadero South/Howard/Steuart | Unit-300/Un...,POINT (-122.39102 37.79161),108,108,1,10,8


In [74]:
df.SESSION_START_DT = pd.to_datetime(df.SESSION_START_DT)
df.SESSION_END_DT = pd.to_datetime(df.SESSION_END_DT)

In [ ]:

import datetime

start_time = datetime.datetime(2023, 4, 15, 0, 0)  # start datetime
end_time = datetime.datetime(2023, 4, 16, 0, 0)  # end datetime

time_diff = datetime.timedelta(minutes=15)  # time difference of 15 minutes

while start_time < end_time:
    interval_end = start_time + time_diff
    start_time += time_diff

    DateRangesOverlap = max(start_time, df.SESSION_START_DT) < min(interval_end, df.SESSION_END_DT)


5           IPS
11          IPS
26          IPS
40          IPS
46          IPS
          ...  
35605       IPS
35606    MacKay
35607       IPS
35613       IPS
35614       IPS
Name: METER_VENDOR, Length: 29313, dtype: object